### ライブラリ読み込み & 初期化

In [5]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
    WebSearchPreviewTool,
    ApproximateLocation
)

# .env 読み込み (.env に AZURE_AI_PROJECT_ENDPOINT などが入っている想定)
load_dotenv()

# Azure AI Projects クライアント
project_client = AIProjectClient(
    endpoint=os.environ["project_endpoint"],
    credential=DefaultAzureCredential(),
)

# OpenAI クライアント
openai_client = project_client.get_openai_client()


### Web検索対応エージェントの作成

In [6]:
agent_name = "web-search-tool-agent-vs"
model_deployment_name = "gpt-4.1"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="あなたは最新の Web 情報を調べて回答できるアシスタントです。",
        tools=[
            WebSearchPreviewTool(),  # ← 接続ID不要、すぐ使える
        ],
    ),
    description="Web検索ができるエージェント",
)

print(f"✅ エージェント作成完了 (名前：{agent.name}, バージョン: {agent.version})")


✅ エージェント作成完了 (名前：web-search-tool-agent-vs, バージョン: 1)


### プロンプト実行

In [7]:
# ====== ここを自由に変えて試せるエリア ======
user_prompt = "今日の東京の天気は？"
# ============================================

response = openai_client.responses.create(
    input=[{"role": "user", "content": user_prompt}],
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print("=== レスポンス ===")
print(response.output_text)


=== レスポンス ===
今日の東京の天気は「晴れ時々曇」。最高気温は21℃、最低気温は8℃で、降水確率は一日を通してほぼ0～10%程度と低く、雨の心配はほとんどありません。全体的に暖かい一日になる見込みです【Yahoo!天気・災害】(https://weather.yahoo.co.jp/weather/jp/13/4410.html)、【tenki.jp】(https://tenki.jp/forecast/3/16/)。
